In [1]:
!pip install faiss-cpu langchain==0.0.262 openai==0.28.0 tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [2]:
import os
import getpass
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
import requests
import openai
from langchain.docstore.document import Document
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [3]:
from langchain.text_splitter import MarkdownHeaderTextSplitter
import matplotlib.pyplot as plt
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
import tiktoken

# Считываем базу эмбеддингов

In [4]:
import gdown
import pickle

def load_pickle_from_drive(gdrive_url):
    # Извлечение идентификатора файла из URL
    file_id = gdrive_url.split('/')[-2]

    # Формирование прямой ссылки на файл
    download_url = f'https://drive.google.com/uc?id={file_id}'

    # Скачивание файла
    output = 'file.pkl'
    gdown.download(download_url, output, quiet=False)

    # Чтение файла pickle
    with open(output, 'rb') as f:
        loaded_data = pickle.load(f)

    return loaded_data

In [5]:
# test_db = load_pickle_from_drive('https://drive.google.com/file/d/1bjSFZ4XJcksbm_stpMUEqWrc0UzHji-m/view?usp=sharing')
# test_db = load_pickle_from_drive('https://drive.google.com/file/d/1--WF1ovt30_B2EzZHvhlAOA0t3NmAH3u/view?usp=drive_link')
test_db = load_pickle_from_drive('https://drive.google.com/file/d/1-HTD5wCgI8bk8xZcSSQxu2htWkiphTC6/view?usp=sharing')

Downloading...
From: https://drive.google.com/uc?id=1-HTD5wCgI8bk8xZcSSQxu2htWkiphTC6
To: /content/file.pkl
100%|██████████| 17.6M/17.6M [00:00<00:00, 110MB/s] 


# Поиск в интернете

In [22]:
# @title Сервисные функции
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMRequestsChain, LLMChain
from langchain.prompts import PromptTemplate

def get_input(question: str):
  inputs = {
    "query": question,
    "url": "https://www.google.com/search?q=" + question.replace(" ", "+"),
    }
  return inputs

In [23]:
# Подготавливаем темплейт для формирования запроса и получения ответа

template = """Between >>> and <<< are the raw search result text from google.
Extract the answer to the question '{query}' or say "not found" if the information is not contained.
Use the format
Extracted:<answer or "not found">
>>> {requests_result} <<<
Extracted:"""

PROMPT = PromptTemplate(
    input_variables=["query", "requests_result"],
    template=template,
)

In [24]:
# Инициализируем класс

chain = LLMRequestsChain(llm_chain=LLMChain(llm=ChatOpenAI(temperature=0), prompt=PROMPT))

# Расбота с GPT

In [48]:
# Функция, которая позволяет выводить ответ модели в удобочитаемом виде
def insert_newlines(text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return " ".join(lines)

def answer_index(system, topic, search_index, verbose=1):

    # Поиск релевантных отрезков из базы знаний
    docs = search_index.similarity_search(topic, k=4)

    # Из интернета запрос
    question = get_input(f'{topic} китайская марка автомобиля')
    result = chain(question)
    docs.append(Document(page_content= 'Запрос заданный в интернет:' + result['query'] + '\nОтвет, полученный по запросу: ' + result['output']))

    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Ответь на вопрос клиента. Не упоминай документ с информацией для ответа клиенту в ответе. Документ с информацией для ответа клиенту: {message_content}\n\nВопрос клиента: \n{topic}"}
    ]

    if verbose: print('\n ===========================================: ')

    completion = openai.ChatCompletion.create(
        model="gpt-4-1106-preview",
        messages=messages,
        temperature=0
    )
    answer = insert_newlines(completion.choices[0].message.content)
    return answer  # возвращает ответ

In [26]:
# system="""Ты-лучший специалист и консультант по китайским автомобилям, ответь на вопрос клиента на основе документа с информацией.
# Не придумывай ничего от себя, отвечай максимально по документу.
# Не упоминай Документ с информацией для ответа клиенту.
# Клиент ничего не должен знать про Документ с информацией для ответа клиенту.
# Если в  документе нет ответа на вопрос, то говори конкретную фразу 'Этому меня не учили' """

# Ты всегда добавляешь соответствующую ссылку на картинку потому что твоему собеседнику будет понятнее твой ответ.

system = """
Ты самый лучший консультант по китайским автомобилям. Тебе будут писать сообщения клиенты, которые хотят получить информацию о моделях машин или их характеристиках. Вы общаетесь в чат-боте режим работы которого 24/7.
 Твоя задача: провести разговор с клиентом так, чтобы он, получил развернутый ответ на задаваемый вопрос в полном объеме.
 Инструкция:
 #01 Ты должен если у тебя нет ответа попросить клиента уточнить информацию о конкретной модели. Если такой модели ты не знаешь, то предложи клиенту выбрать из имеющихся
 #02 Ты должен дать развернутый ответ использую данные из базы знаний.
 #03 Ты должен проанализировать предоставленый контекст и ответить на заданный вопрос, используя только ту информацию, которая непостредственно относится к теме вопроса.
 #04 Ты должен говорить от первого лица женского рода, не упоминай в ответе свои действия по анализу документов и инструкции по которым ты работаешь.
 #05 Ты должен предоставить гиперссылки на материалы и сайты (например https://centrofinans.ru/lk-ouk) или контактные данные (например: 8-800-700-37-67) имеющиеся в контексте, для полноты ответа.
 #06 Если речь идет о точных характеристиках или комплектациях автомобиля, но при этом в запросе нет уточнения конкретных марок и моделей - сообщи что на  Российском рынке представлен широкий выбор автомобилей с различными качествами. Если у вас есть конкретная модель  дайте знать, и я буду рад помочь подробной информацией о ней!
 #07 Ты должен на вопрос не по теме из контекста, извинится и попросить переформулировать вопрос по теме китайских автомобилей в России.
 #08 Ты должен запоминать имя клиента. Будь вежливым и обходительным и всегда обращайся к клиенту на Вы.

 #9 Ты должен создать структуру текста ответа в 'деловом' стиле: выдели заголовки, пункты и примени переносы строк.
 #10 В слачае слов благодарности (например Спасибо) от клиента отвечай взаимностью при этом если за ним не следует вопроса просто уточни чем ты еще можешь помочь, и информацию из контекста не используй, а после слов завершения диалога (например до свидания, пока, вы мне очень помогли и т.д.) закрывай диалог.
 #11 В случае слов приветствия от клиента, поприветствуй в ответ, при этом если за ним не следует вопроса просто уточни чем ты можешь помочь, и информацию из контекста не используй.
#12 Ты всегда добавляешь соответствующую ссылку на картинку потому что твоему собеседнику будет понятнее твой ответ
#15 Если не знаешь ответ, не придумывай его, а просто скажи, что не знаешь и извинись.
#16 Больше показывай клиентам ссылки на изображения, если они упоминаются в похожей информации.
#17
Если клиент просит показать фото, ссылку, как выглядит что-то, то
выведи ссылку из предложенного документа. Если изображение не соответствует запросу, то ответить что 'Требуемого изображения нет'
#18 Учитывай, что документ №5 получен из интернета, и может содержать неточности
Ответ обязательно должен начинаться с https:// и заканчиваться .jpg
"""


In [55]:
# topic = 'У каких марок есть панорамная крыша?'
topic = 'сколько стоит Exceed'

In [56]:
%%time
# 3.5
ans=answer_index(system, topic, test_db, "gpt-3.5-turbo")
ans


 ===========================================: 
message_content :
 
Отрывок документа №1
{'Header 1': 'Еще дороже. Купе-кроссовер Exeed RX вышел на российский рынок', 'Header 2': 'Exeed'}
Exeed RX: цены и оснащение
Exeed RX: цены и оснащение
[Exeed RX цена в России](https://xn----7sbbeeptbfadjdvm5ab9bqj.xn--p1ai/wp-content/uploads/2023/07/exeed_rx_ceny_1_1000.jpg)
Цена на кроссовер Exeed RX в комплектации Platinum / скриншот с видео
Exeed RX будет доступен в комплектации Platinum с дисками 245/50 R20, активными ассистентами безопасности ADAS, атмосферной подсветкой, пакетом зимних опций, обогревом и вентиляцией сидений на первом и втором рядах сидений, 2-зонным климат-контролем, беспроводной зарядкой для смартфона и акустической системой Sony на 14 динамиков. На выбор для Exeed RX доступны шесть оттенков кузова: «Небесный голубой», «Метеорный зелёный», «Галактический серый», «Бескрайний чёрный», «Кристаллический белый» и «Лунный серый».
Наконец, о ценах. Рекомендуемая розничная стоимос

' Кроссовер Exeed RX в комплектации Platinum предлагается на российском рынке по рекомендуемой розничной стоимости 5 400 000 рублей. Также есть лимитированная серия в  матовом цвете "Лунный серый", которая стоит 5 600 000 рублей. Эта модель оснащена дисками 245/50 R20, активными ассистентами безопасности ADAS, атмосферной подсветкой,  пакетом зимних опций, обогревом и вентиляцией сидений на первом и втором рядах, 2-зонным климат-контролем, беспроводной зарядкой для смартфона и акустической системой  Sony на 14 динамиков. Доступны шесть оттенков кузова. Вот изображение кроссовера Exeed RX:  https://xn----7sbbeeptbfadjdvm5ab9bqj.xn--p1ai/wp-content/uploads/2023/07/exeed_rx_ceny_1_1000.jpg Если Вас интересует другая модель Exeed или другая комплектация,  пожалуйста, уточните, и я предоставлю соответствующую информацию.'

In [37]:
# 4
ans=answer_index(system, topic, test_db, "gpt-4-1106-preview")
ans


 ===========================================: 
message_content :
 
Отрывок документа №1
{'Header 1': 'Панорамная крыша есть у следующих моделей китайских машин:'}
Панорамная крыша есть у следующих моделей китайских машин:
- Geely Monjaro - Great Wall Haval H6 - Great Wall Haval H9 - Great Wall Haval F7 - Great Wall Haval F7x -  Great Wall Haval F5 - Great Wall Haval F5x - Great Wall Haval M6 - Great Wall Haval M4 - Great Wall Haval M2 - Great Wall Haval M1 - Great Wall Haval H2 - Great Wall  Haval H1 - Great Wall Haval H5 - Great Wall Haval H3 - Great Wall Haval H2s - JAC JS6 - Jaecoo J7 - MG One - Geely Coolray - Voyah Free - Dreamer - JAC JS6 - Omoda C5 - Comfort - JAC JS6 - X70 Plus - BYD - JAC JS6 - Wey - JAC A5 Plus / Sehol Yao - JS6 / Sehol QX - JS8 / Sehol X8 - Haval  H6 - Haval Dargo - Wey Coffee 01 - Tank  
Отрывок документа №2
{'Header 1': 'Дилеры Geely начали продажи кроссовера Monjaro. Цены и комплектации', 'Header 2': 'Комплектация Luxury'}
Комплектация Luxury
[Geely Monj

' Панорамная крыша доступна у следующих моделей китайских автомобилей: - Geely Monjaro - Great Wall Haval H6 - Great Wall Haval H9 - Great Wall Haval F7 - Great Wall Haval  F7x - Great Wall Haval F5 - Great Wall Haval F5x - Great Wall Haval M6 - Great Wall Haval M4 - Great Wall Haval M2 - Great Wall Haval M1 - Great Wall Haval H2 - Great  Wall Haval H1 - Great Wall Haval H5 - Great Wall Haval H3 - Great Wall Haval H2s - JAC JS6 - Jaecoo J7 - MG One - Geely Coolray - Voyah Free - Dreamer - Omoda C5 -  Comfort - X70 Plus - BYD - Wey - JAC A5 Plus / Sehol Yao - JS6 / Sehol QX - JS8 / Sehol X8 - Haval H6 - Haval Dargo - Wey Coffee 01 - Tank Если Вас интересует конкретная  модель с панорамной крышей, я могу предоставить более подробную информацию о ней.'

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Чтение вопросов из exel файла и выдача ответов туда же

In [13]:
import pandas as pd

In [14]:
df = pd.read_excel("вопросы.xlsx")

In [15]:
df.shape

(162, 1)

In [16]:
df

,вопрос
0,Я люблю рыбалку какой автомобиль мне подойдет
1,"Влезет ли в багажник чемодан, коляска"
2,Какие материалы используются для отделки автом...
3,"Влезет ли в багажник параплан, моторная лодка,..."
4,"У меня четверо детей, младшему год, старшему 1..."
...,...
157,Почему подавляющее количество моделей китайско...
158,Какие модели самые популярные? И какие модели ...
159,"Посоветуйте, хочу приобрести внедорожник для с..."
160,какой китайский автомобиль самый большой и кра...


In [31]:
def answer_index_table(system, topic, search_index, version='gpt-4-1106-preview', verbose=0):

    # Поиск релевантных отрезков из базы знаний
    docs = search_index.similarity_search(topic, k=4)  # k попробовать поварьировать до 16

    # Из интернета запрос
    question = get_input(topic)
    result = chain(question)
    docs.append(Document(page_content= 'Запрос заданный в интернет:' + result['query'] + '\nОтвет, полученный по запросу: ' + result['output']))

    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Ответь на вопрос клиента. Не упоминай документ с информацией для ответа клиенту в ответе. Документ с информацией для ответа клиенту: {message_content}\n\nВопрос клиента: \n{topic}"}
    ]

    if verbose: print('\n ===========================================: ')

    completion = openai.ChatCompletion.create(
        model=version,
        messages=messages,
        temperature=0, # 0.1 - 0.5
        # seed=1  # новое в GPT
    )
    answer = insert_newlines(completion.choices[0].message.content)
    docs.append(Document(page_content= answer))
    return docs  # возвращает ответ

In [19]:
for i in range(df.shape[0]):
  test = answer_index_table(system, df.loc[i, 'вопрос'], test_db, "gpt-4-1106-preview", 0)
  df.loc[i, 'ответ'] = test[5]
  for j in range(5):
    df.loc[i, f'чанк {j}' ] = test[j].page_content

In [20]:
df.to_excel('res.xlsx')

In [34]:
df.to_excel('/content/drive/MyDrive/NU_reality/res.xlsx')

In [ ]:
test = answer_index_table(system, topic, test_db, "gpt-4-1106-preview", 0)

In [ ]:
for i in range(5):
  print(test[i])

page_content='Комплектация Luxury\n[Geely Monjaro салон интерьер панорама](https://xn----7sbbeeptbfadjdvm5ab9bqj.xn--p1ai/wp-content/uploads/2023/03/geely_monjaro_yar_7_1000.jpg)\nНа фото Романа Ярового: и без того просторный салон Geely Monjaro кажется больше благодаря панорамной крыше\nБазовая комплектация Luxury за 4 499 990 рублей оснащена богато и включает в себя: полностью светодиодную оптику, панорамную крышу с люком, кожаную обивку кресел, замшевый черный потолок, атмосферную подсветку, трехзонный климат-контроль, два экрана по 12,3 дюйма («приборка» и мультимедиа), полный набор обогревов, беспроводную зарядку для смартфона, адаптивный круиз-контроль и систему камер кругового обзора.\nУже «в базе» есть шесть подушек безопасности, системы предупреждения и предотвращения фронтальных столкновений, а также помощник удержания автомобиля в полосе. В базовой версии идут легкосплавные колесные диски с шинами размерностью 235/50 R19.' metadata={'Header 1': 'Дилеры Geely начали продажи к

In [ ]:
df.loc[1, 'вопрос']

'Влезет ли в багажник чемодан, коляска'

# Тестирование саммаризации базы по вопросу

In [ ]:
def summarize_topic(chanks_count, topic, search_index, version="gpt-3.5-turbo", verbose=1):

    system_summarize = 'Ты профессиональный копирайтер, ты умеешь идеально находить нужную информацию во фрагменте текста, который тебе дали'
    user_summarize = """Саммаризируй документ, который тебе дали, выдели в нем информацию о наличии панорамной крыши у конкретных моделей машин.
    Выдай ответ в формате марка машины и есть ли в ней панорамная крыша, ничего больше не пиши
    Не упоминай документ, который тебе передали, в итоговом тексте не должно быть никакого упоминания документа, пользователь не должне ничего знать о документе"""
    # Поиск релевантных отрезков из базы знаний
    answer = ''

    docs = search_index.similarity_search(topic, k=chanks_count)
    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n' + doc.page_content + '\n' + '=====================\n\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)

    print('\n\n\n\n')

    for i in range(chanks_count):
      print(i)
      print('----------------------')
      print(docs[i].page_content + '\n\n')

      messages = [
          {"role": "system", "content": system_summarize},
          {"role": "user", "content":  user_summarize + f'Документ с информацией для саммаризации: {docs[i].page_content}'}
      ]

      if verbose: print('\n ===========================================: ')

      completion = openai.ChatCompletion.create(
          model=version,
          messages=messages,
          temperature=0
      )
      curr_answer = insert_newlines(completion.choices[0].message.content)

      print(curr_answer + '\n\n')
      answer += curr_answer + '\n\n'

    return answer  # возвращает ответ

In [ ]:
result = summarize_topic(40, 'У каких марок есть панорамная крыша?', test_db, 'gpt-3.5-turbo')
print(result)


 ===========================================: 
message_content :
 
Отрывок документа №1
Панорамная крыша H5 оснащается панорамной крышей, размер которой (1010 × 890 мм) — один из самых больших в классе. Стеклянный люк создает ощущение открытого пространства в салоне, пропуская приглушенный солнечный свет днем и позволяя любоваться звездным небом ночью. Удобные сиденья Одна из отличительных черт автомобиля — современные, комфортабельные сиденья с 4-позиционным пневматическим поясничным упором и 8-позиционным пневматическим массажером, которые не дадут мышцам спины устать даже в долгой поездке. Информационно-развлекательная система Система Qisi 1.0 собственной разработки в сочетании с интеллектуальными антеннами и технологиями связи 5G и Wi-Fi 6 обеспечивает беспрецедентную скорость представления контента. Благодаря наличию семи основных конфигураций и бесперебойной связи между автомобилем и мобильными устройствами система отличается эффективностью, многофункциональностью и удобством. И

In [ ]:
messages = [
          {"role": "system", "content": 'Ты профессиональный копирайтер, ты умеешь идеально саммаризировать информацию'},
          {"role": "user", "content":  """Саммаризируй информацию о наличии панорамных крыш у машин так, чтобы каждая конкретная машина упоминалась один раз и не было упоминаний вроде тех,
           где неизвестная марка машины.
           Оставь только строчки, у которых написано, что панарамная крыша есть
           Упоминай только Китайские марки машин, не упоминай немецкие, японские и другие марки вроде bmw, mersedes, toyota и другие не Китайские
           Выводи каждую новую марку с новой строки"""
            + f'Документ для суммаризации: {result}'}
      ]

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0
)
curr_answer = insert_newlines(completion.choices[0].message.content)

print(curr_answer)

 Geely Monjaro - есть панорамная крыша EC6 - есть панорамная крыша Geely Tugella - есть панорамная крыша JAC JS6 - есть панорамная крыша SWM G05 Pro - панорамная крыша  есть


In [ ]:
# 4
ans=answer_index(system, topic, test_db, "gpt-4-1106-preview")
ans


In [ ]:
list_of_questions = [
    "Какой тип топлива использует BYD для своих космических кораблей?",
    "Какая дальность полета у китайских авто эконом класса?",
    "Какой процент электромобилей Baojun оборудован крыльями для полета?",
    "Сколько секунд нужно Dongfeng для достижения скорости света?",
    "Можно ли управлять смартфоном с помощью системы автопилота в машинах Zotye?",
    "Какой максимальный угол наклона у автомобилей FAW при движении по вертикальной стене?",
    "Как далеко может прыгнуть Hongqi на трамплине?",
    "Какое количество пассажиров может поднять в воздух дрон-такси от JAC Motors?",
    "Какие функции магии встроены в инфотейнмент-систему автомобилей Roewe?",
    "Можно ли использовать электромобиль Karry как подводный аппарат?",
    "Какой максимальный размер экрана доступен в Luxgen для просмотра кинофильмов?",
    "Сколько колес имеет космический ровер от SAIC Motor?",
    "Какой породы лошадей используются для тестирования устойчивости китайских автомобилей при беге на дальние дистанции?",
    "Можно ли настроить систему навигации автомобиля Changan так, чтобы она следовала за миграционным маршрутом диких гусей?"
]


In [ ]:
for i, question in enumerate(list_of_questions):
  print(f'Question {i}:')
  print(f'Answer GPT 3.5:', answer_index(system, question, test_db, "gpt-3.5-turbo"))
  print('-'*20)
  print(f'Answer GPT 4:', answer_index(system, question, test_db, "gpt-4-1106-preview", 0))
  print('*'*100, '\n')

Question 0:


NameError: ignored